#### Imports

In [5]:
import pandas as pd
import requests
import time
import os

from bs4 import BeautifulSoup

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium import webdriver

from dotenv import load_dotenv
load_dotenv()

True

#### Scrapping Linkedin Jobs

In [27]:
url = 'https://www.linkedin.com'
driver = webdriver.Chrome()
driver.get(url)

time.sleep(2)

email_box = driver.find_element(By.ID, "session_key")
password_box = driver.find_element(By.ID, "session_password")

# 2. Declare variables
email = os.getenv("email")
password = os.getenv("password")

# 3. Send the text
email_box.send_keys(email)
password_box.send_keys(password)

# 4. Hit enter
password_box.send_keys(Keys.RETURN)

time.sleep(2)
# Navigate to jobs tab
jobs_button = driver.find_element(By.XPATH, '/html/body/div[6]/header/div/nav/ul/li[3]/a')
jobs_button.click()

time.sleep(2)
# Filter for job title
job_box = driver.find_element(By.XPATH, '/html/body/div[6]/header/div/div/div/div[2]/div[2]/div/div/input[1]')
time.sleep(1)
job_box.send_keys('Ciencia de datos')
job_box.send_keys(Keys.ENTER)

time.sleep(2)
# Filter for location
location_box = driver.find_element(By.XPATH, '/html/body/div[6]/header/div/div/div/div[2]/div[2]/div/div/input[1]')
time.sleep(1)
location_box.clear()
time.sleep(3)
location_box.send_keys('Estados Unidos')
location_box.send_keys(Keys.ENTER)

# Job scrapping with bs4
html = driver.execute_script("return document.body.outerHTML;") # Javascript script

In [37]:
soup = BeautifulSoup(html, 'html.parser')
container = soup.find_all('ul', {'class': 'scaffold-layout__list-container'})
container = container[0]
with open('../data/job_lists.txt', 'w') as file:
    file.write(str(container))

In [57]:
def get_job_card_info(el):
    print(el)
    job = {}
    job['job_id'] = el.get('data-job-id')
    job['job_title'] = el.find('a').getText().strip()
    job['company'] = el.find_all('span', {'class': 'job-card-container__primary-description'})[0].getText().strip()
    job['location'] = el.find_all('li', {'class': 'job-card-container__metadata-item'})[0].getText().strip()
    return job

def build_linkedin_dataset(list_):
    jobs = []
    for card in list_[1:3]:
        jobs.append(get_job_card_info(card))
    print(jobs)

list_ = container.find_all('div', {'class': 'job-card-container'})
list_
jobs = build_linkedin_dataset(list_)

<div aria-hidden="false" class="job-card-container relative job-card-list job-card-container--clickable job-card-list--underline-title-on-hover jobs-search-two-pane__job-card-container--viewport-tracking-1" data-job-id="3769019036" data-view-name="job-card">
<div class="job-card-list__entity-lockup artdeco-entity-lockup artdeco-entity-lockup--size-4 ember-view" id="ember606">
<div class="mr1 job-card-list__logo artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view" id="ember607" type="square">
<div class="ivm-image-view-model job-card-list__logo-ivm">
<div class="ivm-view-attr__img-wrapper display-flex">
<!-- --> <img alt="Logotipo de Coders Data" class="ivm-view-attr__img--centered EntityPhoto-square-4 evi-image lazy-image ember-view" height="56" id="ember608" loading="lazy" src="https://media.licdn.com/dms/image/C510BAQFkOlS-dCHocA/company-logo_100_100/0/1630584787684/coders_data_logo?e=1709164800&amp;v=beta&amp;t=pHcwjpqMbU3hobxVCaizevrN9QrY_eL0zL51OXwDsy